In [14]:
import time
import os
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ

In [15]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/Kate's/"
csv_filename = 'raw_velocity_data_JJ.csv'

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_filename)
df = pd.DataFrame(df)

enable_photoZ_interaction = False

In [16]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    aPhz.prepare_photoZ()

In [52]:
for subdir, dirs, files in os.walk(data_dir):
    zda_file = None
    roi_files = []
    roi_files_spare = []
    
    # locate important file names
    for f in files:
        if f.endswith('.zda'):
            zda_file = f
        if f.startswith('rois_'):
            if f.endswith('laminar.dat'):
                roi_files.append(f)
            else:
                roi_files.append(f)
        if zda_file is not None and len(roi_files) == 2:
            break
    if len(roi_files) < 1:
        roi_files = roi_files_spare
        if len(roi_files) > 0:
            roi_files = [roi_files[0]]
        
    if zda_file is not None:
        rec_id = zda_file.split('.')[0]
        # alias the rec_id
        if '_aka_' in subdir:
            rec_id = subdir.split('_aka_')[0][-8:].replace("-", "_")
            print("aliased:", rec_id)
            
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            aPhz.open_zda_file(zda_file)
        
        date = subdir.split("/")[-1].split('\\')[0].replace('-', '_')
        date = date[5:] + '_' + date[:4]
        print(subdir)
        for roi_file in roi_files:
            print(roi_file)
            
            # open roi file
            if enable_photoZ_interaction:
                aPhz.open_roi_file(roi_file)
            
            # Now load the data from the CSV needed to know the monosynaptic window
            lam_type = None
            if 'interlaminar' in roi_file and 'interlaminar' in subdir:
                lam_type = ['Inter', 'Inter_L23', 'Inter_L5']
            elif 'intralaminar' in roi_file or 'intralaminar' in subdir:
                lam_type = ['Intra']
            elif 'interlaminar' in subdir or 'interlaminar' in roi_file:
                lam_type = ['Inter', 'Inter_L23', 'Inter_L5']
            else:
                raise Exception("What is the laminar type? Unknown")
            mono_window = df[(df['Date'] == date) 
                             & (df['Laminar'].isin(lam_type))
                             & (df['Slice_Loc_Run'] == rec_id)]
            mono_window = pd.DataFrame(mono_window,
                                       columns = ['Date', 'Laminar',
                                                'Mono_Resp_Start_Time',
                                                'Mono_Resp_End_Time'])
            if mono_window.size < 1:
                raise Exception("(No row) Check laminar value " + " ".join(lam_type) + " for date " + date + " rec " + rec_id)
                continue
            mono_window = mono_window.drop_duplicates()
            if mono_window.shape[0] != 1:
                print(mono_window, mono_window.shape)
                raise Exception("Check laminar value " + " ".join(lam_type) + " for date " + date + " rec " + rec_id)
                
            
            

C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar
rois_01_to_20.dat
rois_01_to_20_interlaminar.dat
C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-27\03-01-06_L4_intralaminar
rois_01_to_10.dat
rois_01_to_10_interlaminar.dat
C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-13\02-01-02_L4_intralaminar
rois_01_to_26.dat
aliased: 03_01_08
C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-28\03-01-08_aka_02-01-08_L23_intralaminar
rois_01_to_19.dat
aliased: 04_01_01
C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-28\04-01-01_aka_03-01-01_L4_interlaminar
rois_01_to_14.dat
aliased: 06_01_01
C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-28\06-01-01_aka_part2_03-01-01_L4_interlaminar
rois_01_to_14.dat
aliased: 08_01_04
C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-28\08-01-04_aka_part2_05-01-04_L4_interlaminar
rois_01_to_19.dat
aliased: 10_01_01
C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-28\10-01-01_aka_part2_07-01-01_L4_interlaminar
rois_01_to_13.dat
aliased: 11_01_06
C:/U